In [ ]:
import pydicom
import numpy as np
import os
from collections import deque

def hu_degerine_donustur(dicom_goruntusu):
    goruntu = dicom_goruntusu.pixel_array
    egim = dicom_goruntusu.RescaleSlope
    kesme_noktasi = dicom_goruntusu.RescaleIntercept
    goruntu_hu = goruntu * egim + kesme_noktasi
    return goruntu_hu

def klasorden_dcm_okuma(klasor_yolu):
    try:
        dcm_dosyalari = [os.path.join(klasor_yolu, f) for f in os.listdir(klasor_yolu) if f.endswith('.dcm')]
        
        if not dcm_dosyalari:
            print("Belirtilen klasörde DICOM dosyası bulunamadı.")
            return None

        dicom_dosyalari = [pydicom.dcmread(f) for f in dcm_dosyalari]
        dicom_dosyalari.sort(key=lambda x: float(x.ImagePositionPatient[2]))

        hu_goruntuleri = [hu_degerine_donustur(dcm) for dcm in dicom_dosyalari]
        volume_3d = np.stack(hu_goruntuleri, axis=0)
        
        return volume_3d
    
    except Exception as e:
        print(f"Bir hata oluştu: {e}")
        return None

def find_region_center(ct_volume, start_coords, hu_range):
    z_max, y_max, x_max = ct_volume.shape
    start_x, start_y, start_z = start_coords

    if not (0 <= start_z < z_max and 0 <= start_y < y_max and 0 <= start_x < x_max):
        print("Hata: Başlangıç koordinatları hacmin sınırları dışında.")
        return None
    
    min_hu, max_hu = hu_range
    
    if not (min_hu <= ct_volume[start_z, start_y, start_x] <= max_hu):
        print(f"Hata: Başlangıç koordinatındaki HU değeri ({ct_volume[start_z, start_y, start_x]}) belirtilen aralıkta ({hu_range}) değil.")
        return None
        
    queue = deque([(start_z, start_y, start_x)])
    mask = np.zeros_like(ct_volume, dtype=bool)
    mask[start_z, start_y, start_x] = True
    
    neighbors = [(-1, 0, 0), (1, 0, 0), (0, -1, 0), (0, 1, 0), (0, 0, -1), (0, 0, 1)]
    
    while queue:
        z, y, x = queue.popleft()
        
        for dz, dy, dx in neighbors:
            nz, ny, nx = z + dz, y + dy, x + dx
            
            if (0 <= nz < z_max and 0 <= ny < y_max and 0 <= nx < x_max and not mask[nz, ny, nx]):
                if min_hu <= ct_volume[nz, ny, nx] <= max_hu:
                    mask[nz, ny, nx] = True
                    queue.append((nz, ny, nx))

    region_points = np.argwhere(mask)
    if region_points.size == 0:
        print("Hata: Belirtilen aralıkta bir bölge bulunamadı.")
        return None
        
    center_z, center_y, center_x = np.mean(region_points, axis=0)

    center_coords = (int(round(center_x)), int(round(center_y)), int(round(center_z)))
    
    return center_coords

dicom_klasor_yolu = r'D:\staj\AI\CT_Dicom\patient2_2'

##Second
##baslangic_koordinati = (345, 323, 127)
##baslangic_koordinati = (349, 324, 127)
##baslangic_koordinati = (351, 317, 128)
##First
##baslangic_koordinati = (354, 327, 127)
##Patient_1
baslangic_koordinati = (346, 313, 131)

hu_araligi = (-200, 100)

ct_volume = klasorden_dcm_okuma(dicom_klasor_yolu)

if ct_volume is not None:
    merkez_koordinat = find_region_center(ct_volume, baslangic_koordinati, hu_araligi)
    
    if merkez_koordinat:
        x, y, z = merkez_koordinat
        print(f"Belirtilen HU aralığındaki (HU > {hu_araligi[0]} ve HU < {hu_araligi[1]}) bölgenin merkez koordinatları: (x={x}, y={y}, z={z})")
    else:
        print("Merkez koordinatı bulunamadı.")
else:
    print("3D hacim oluşturulamadı. İşlem sonlandırılıyor.")

Belirtilen HU aralığındaki (HU > -200 ve HU < 100) bölgenin merkez koordinatları: (x=353, y=306, z=129)


In [18]:
import pydicom
import numpy as np
import os

def hu_degerine_donustur(dicom_goruntusu):
    goruntu = dicom_goruntusu.pixel_array
    egim = dicom_goruntusu.RescaleSlope
    kesme_noktasi = dicom_goruntusu.RescaleIntercept
    goruntu_hu = goruntu * egim + kesme_noktasi
    return goruntu_hu

def klasorden_dcm_okuma(klasor_yolu):
    dcm_dosyalari = [os.path.join(klasor_yolu, f) for f in os.listdir(klasor_yolu) if f.endswith('.dcm')]
    if not dcm_dosyalari:
        print("Belirtilen klasörde DICOM dosyası bulunamadı.")
        return None

    dicom_dosyalari = [pydicom.dcmread(f) for f in dcm_dosyalari]
    dicom_dosyalari.sort(key=lambda x: float(x.ImagePositionPatient[2]))

    hu_goruntuleri = [hu_degerine_donustur(dcm) for dcm in dicom_dosyalari]
    volume_3d = np.stack(hu_goruntuleri, axis=0)
    
    return volume_3d

dicom_klasor_yolu = r"D:\staj\AI\CT_Dicom\patient2_1"

koordinat = (346, 313, 131)

ct_volume = klasorden_dcm_okuma(dicom_klasor_yolu)

if ct_volume is not None:
    x, y, z = koordinat
    
    if z < ct_volume.shape[0] and y < ct_volume.shape[1] and x < ct_volume.shape[2]:
        hu_degeri = ct_volume[z, y, x]
        print(f"Belirtilen ({x}, {y}, {z}) koordinatındaki HU değeri: {hu_degeri}")
    else:
        print("Belirtilen koordinatlar 3D hacmin sınırları dışında.")

Belirtilen (346, 313, 131) koordinatındaki HU değeri: -690.0


In [ ]:
import pydicom

file1 = r"D:\staj\AI\CT_Dicom\First\274393760.dcm"
file2 = r"D:\staj\AI\CT_Dicom\Second\286267012.dcm"

dcm1 = pydicom.dcmread(file1)
dcm2 = pydicom.dcmread(file2)

def meta_info(dcm):
    return {
        "PixelSpacing": getattr(dcm, "PixelSpacing", None),
        "SliceThickness": getattr(dcm, "SliceThickness", None),
        "ImagePositionPatient": getattr(dcm, "ImagePositionPatient", None),
        "ImageOrientationPatient": getattr(dcm, "ImageOrientationPatient", None),
        "RescaleIntercept": getattr(dcm, "RescaleIntercept", None),
        "RescaleSlope": getattr(dcm, "RescaleSlope", None),
    }

print("Dosya 1:", meta_info(dcm1))
print("Dosya 2:", meta_info(dcm2))
